In [6]:
import requests
import pandas as pd
from http import HTTPStatus
import dashscope
import re

In [7]:
def preprocess_gsm8k(data_path, num_records=50):
    df = pd.read_parquet(data_path)
    df = df.head(num_records)
    questions = df['question'].tolist()
    answers = df['answer'].str.split('####').str[1].tolist()
    return questions, answers

def preprocess_commonsenseqa(data_path, num_records=50):
    df = pd.read_parquet(data_path)
    df = df.head(num_records)
    questions = df['question'].tolist()

    def format_choices(choice_dict):
        combined_choices = ' '.join([f"{label}: {text}" for label, text in zip(choice_dict['label'], choice_dict['text'])])
        return combined_choices

    formatted_choices = df['choices'].apply(format_choices).tolist()
    input_texts = [f"{q} Choices: {c}" for q, c in zip(questions, formatted_choices)]
    answer_key = df['answerKey'].tolist()
    return input_texts, answer_key


In [8]:
num_records = 50
gsm8k_questions, gsm8k_answers = preprocess_gsm8k('./cot/gsm8k_test.parquet', num_records)
commonsenseqa_questions, commonsenseqa_answers = preprocess_commonsenseqa('./cot/commonsense_qa_test.parquet',
                                                                          num_records)

In [9]:
dashscope.api_key = "sk-01944338238149dda0a7b55d5b8e175e"

def call_with_messages(content):
    messages = [{'role': 'user', 'content': content}]
    response = dashscope.Generation.call(
        'qwen-plus',
        messages=messages,
        result_format='message',
        max_tokens=500
    )
    if response.status_code == HTTPStatus.OK:
        return response.output.choices[0]['message']['content']
    else:
        print('Error from DashScope:', response.status_code)
        return None

def qwen_infer(prompt):
    return call_with_messages(prompt)


In [10]:
# 标准模板实现
def infer_cot(questions, is_arithmetic=True):
    results = []
    for question in questions:
        X0 = f"Q: {question}. A: Let's think step by step."
        print(f"Q: {question}.")
        print("A: Let's think step by step.")
        Z = qwen_infer(X0)
        print(Z)
        if is_arithmetic:
            prompt = f"{X0} {Z} Therefore, the answer (arabic numerals) is"
            print("Therefore, the answer (arabic numerals) is")
        else:
            prompt = f"{X0} {Z} Therefore, among A through E, the answer is"
            print("Therefore, among A through E, the answer is")
        result = qwen_infer(prompt)
        print(result)
        print("----------------------------------------------------")
        results.append(result)
    return results

# 测试输出
print("Arithmetic problems infer starting" + "-" * 25)
arithmetic_results = infer_cot(gsm8k_questions)

print("Commonsense problems infer starting" + "-" * 24)
commonsense_results = infer_cot(commonsenseqa_questions, is_arithmetic=False)

Arithmetic problems infer starting-------------------------
Q: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?.
A: Let's think step by step.
Sure, let's break it down:

1. Janet's ducks lay 16 eggs per day.
2. She eats 3 eggs for breakfast.
3. She uses 4 eggs to bake muffins.

First, we subtract the eggs she uses from the total number laid:

Total eggs - Eggs for breakfast - Eggs for muffins = Eggs for sale
16 - 3 - 4 = 9 eggs

Now, she sells these remaining 9 eggs at the farmers' market for $2 each.

Total earnings = Number of eggs for sale * Price per egg
Total earnings = 9 * $2

Let's calculate that:

Total earnings = $18

So, Janet makes $18 every day at the farmers' market.
Therefore, the answer (arabic numerals) is
18.
--------------------------------

In [11]:
def extract_answer(prediction, is_arithmetic=True):
    if prediction:
        match = re.search(r'\b(0|[1-9]\d*)\b' if is_arithmetic else r'[A-Z]', prediction)
        if match:
            return int(match.group(0)) if is_arithmetic else match.group(0)
    return None

def calculate_accuracy(predictions, true_answers, is_arithmetic=True):
    correct = sum(
        1 for pred, true in zip(predictions, true_answers)
        if extract_answer(pred, is_arithmetic) == (int(true) if is_arithmetic else true)
    )
    return correct / len(predictions) if predictions else 0

# 计算准确率
arithmetic_accuracy = calculate_accuracy(arithmetic_results, gsm8k_answers)
commonsense_accuracy = calculate_accuracy(commonsense_results, commonsenseqa_answers, is_arithmetic=False)

print(f"Arithmetic Accuracy: {arithmetic_accuracy * 100:.2f}%")
print(f"Commonsense Accuracy: {commonsense_accuracy * 100:.2f}%")

Arithmetic Accuracy: 86.00%
Commonsense Accuracy: 82.00%


In [13]:
# 计算算术问题的准确率
arithmetic_true_answers = [int(ans) for ans in gsm8k_answers]
arithmetic_accuracy = calculate_accuracy(arithmetic_results, arithmetic_true_answers)
print(f"Arithmetic accuracy: {arithmetic_accuracy:.2%}")

# 计算常识问答的准确率
commonsense_accuracy = calculate_accuracy(commonsense_results, commonsenseqa_answers, is_arithmetic=False)
print(f"Commonsense accuracy: {commonsense_accuracy:.2%}")

Arithmetic accuracy: 86.00%
Commonsense accuracy: 82.00%


In [15]:
# 定义不同的模板和触发句
templates = [
    {"trigger1": "Q: [X]. A: Let's think step by step.", "trigger2": "Therefore, the answer (arabic numerals) is"},
    {"trigger1": "Q: [X]. A: For the multiple choice question above, let's think through it step by step. We'll evaluate each option and determine the most logical answer based on the context.", "trigger2": "Therefore, the correct answer among A, B, C, D, and E is"},
    {"trigger1": "Q: [X]. A: To find the answer, we need to piece together the information.", "trigger2": "Hence, the result is"},
    {"trigger1": "Q: [X]. A: Let’s approach this problem systematically.", "trigger2": "Consequently, we get the answer as"},
    {"trigger1": "Q: [X]. A: First, let's consider the components of the problem.", "trigger2": "Thus, the calculated answer is"},
    {"trigger1": "Q: [X]. A: Let’s analyze the information step by step.", "trigger2": "Thus, we can infer that"},
    {"trigger1": "Q: [X]. A: To explain this, we need to consider how the components interact.", "trigger2": "Hence, the underlying principle is"},
    {"trigger1": "Q: [X]. A: To find the answer, we need to break down the problem.", "trigger2": "Hence, the final result is:"},
    {"trigger1": "Q: [X]. A: Let’s approach this by examining each part of the problem carefully.", "trigger2": "The resulting answer is:"}
]

In [16]:
def infer_arithmetic(questions, trigger1, trigger2):
    results = []
    for question in questions:
        X0 = trigger1.replace("[X]", question)
        Z = qwen_infer(X0)
        prompt = f"{X0} {Z} {trigger2}"
        result = qwen_infer(prompt)
        results.append(result)
    return results


def infer_commonsense(questions, trigger1, trigger2):
    results = []
    for question in questions:
        X0 = trigger1.replace("[X]", question)
        Z = qwen_infer(X0)
        prompt = f"{X0} {Z} {trigger2}"
        result = qwen_infer(prompt)
        results.append(result)
    return results

In [18]:
# 计算准确率
i = 0
for tmpl in templates:
    print(f"Template {i}" + "=" * 15)
    i += 1 
    arithmetic_results = infer_arithmetic(gsm8k_questions, tmpl['trigger1'], tmpl['trigger2'])
    arithmetic_accuracy = calculate_accuracy(arithmetic_results, gsm8k_answers)
    print(f"Arithmetic accuracy: {arithmetic_accuracy:.2%}")

    commonsense_results = infer_commonsense(commonsenseqa_questions, tmpl['trigger1'], tmpl['trigger2'])
    commonsense_accuracy = calculate_accuracy(commonsense_results, commonsenseqa_answers, is_arithmetic=False)
    print(f"Commonsense accuracy: {commonsense_accuracy:.2%}")

Template 0===============
Arithmetic accuracy: 88.00%
Commonsense accuracy: 52.00%
Template 1===============
Arithmetic accuracy: 58.00%
Commonsense accuracy: 84.00%
Template 2===============
Arithmetic accuracy: 88.00%
Commonsense accuracy: 82.00%
Template 3===============
Arithmetic accuracy: 80.00%
Commonsense accuracy: 74.00%
Template 4===============
Arithmetic accuracy: 84.00%
Commonsense accuracy: 84.00%
Template 5===============
Arithmetic accuracy: 86.00%
Commonsense accuracy: 26.00%
Template 6===============
Arithmetic accuracy: 16.00%
Commonsense accuracy: 16.00%
Template 7===============
Arithmetic accuracy: 84.00%
Commonsense accuracy: 70.00%
Template 8===============
Arithmetic accuracy: 76.00%
Commonsense accuracy: 78.00%
